In [1]:
# quick ETL demo analytics
import sqlite3
import pandas as pd
conn = sqlite3.connect('data/epic_synth.db')
# counts
counts = {t: conn.execute(f"select count(*) from {t}").fetchone()[0] for t in ['person','observation','condition','visit_occurrence','measurement','variant_pathogenic','variant_vrs'] if True}
print(counts)
# top genes by pathogenic variant count
q = "select GeneSymbol, count(*) as c from variant_pathogenic group by GeneSymbol order by c desc limit 10"
top_genes = pd.read_sql(q, conn)
print('\nTop genes by pathogenic variants:')
print(top_genes)

# tiny ML demo skeleton: build a table of gene-level features (pathogenic variant counts) and a dummy target
vg = pd.read_sql('select GeneSymbol, count(*) as pathogenic_count from variant_pathogenic group by GeneSymbol', conn)
# simple target: genes with > 50 pathogenic variants flagged as 1
vg['target'] = (vg['pathogenic_count'] > 50).astype(int)
print('\nML sample head:')
print(vg.head())
conn.close()

{'person': 116, 'observation': 64701, 'condition': 4507, 'visit_occurrence': 6277, 'measurement': 50621, 'variant_pathogenic': 457731, 'variant_vrs': 421145}

Top genes by pathogenic variants:
  GeneSymbol      c
0      BRCA2  10505
1        NF1   9234
2      BRCA1   8171
3        ATM   6230
4       FBN1   4794
5        APC   4656
6        DMD   4556
7       MSH6   3990
8       MSH2   3924
9       MLH1   3284



ML sample head:
                                          GeneSymbol  pathogenic_count  target
0                                                  -               706       1
1  A-GAMMA3'E;BGLT3;HBE1;HBG1;HBG2;HS-E1;LOC10609...                 1       0
2                                             A4GALT                 3       0
3                                     A4GALT;ARFGAP3                 1       0
4                                               AAAS               146       1


In [2]:
# Variant index and patient-join demo
# Create a compact variant_index table from variant_vrs and a small synthetic patient_variant mapping for demo joins.
import sqlite3
conn = sqlite3.connect('data/epic_synth.db')
cur = conn.cursor()
# create variant_index: distinct vrs_id, gene, clinical_count
cur.execute('''
CREATE TABLE IF NOT EXISTS variant_index AS
SELECT vrs_id, GeneSymbol, count(*) as n_occurrences
FROM variant_vrs
GROUP BY vrs_id, GeneSymbol
''')
conn.commit()
# create a tiny patient_variant mapping for demo: sample first 100 vrs_ids and assign randomly to patients
cur.execute('CREATE TABLE IF NOT EXISTS patient_variant (person_id INTEGER, vrs_id TEXT)')
cur.execute('DELETE FROM patient_variant')
# pick up to 100 vrs_ids
v = cur.execute('select vrs_id from variant_vrs limit 100').fetchall()
persons = [r[0] for r in cur.execute('select rowid from person').fetchall()]
for i, (vrs,) in enumerate(v):
    pid = persons[i % len(persons)]
    cur.execute('insert into patient_variant (person_id,vrs_id) values (?,?)', (pid, vrs))
conn.commit()
# example join: list persons with pathogenic variant counts via variant_index
q = '''
select p.rowid as person_id, p.* , vi.n_occurrences
from person p
join patient_variant pv on pv.person_id = p.rowid
join variant_index vi on vi.vrs_id = pv.vrs_id
limit 20
'''
import pandas as pd
print(pd.read_sql(q, conn))
conn.close()

   person_id                             person_id              given_name  \
0       1081  029e6920-aeb4-331a-5ae0-661315e6f25c          Erin498 Sid118   
1       1072  02a0367f-9518-1b16-9930-76bf4eed7ca3        Al123 Micheal721   
2       1142  03bdd639-5a5d-3712-0687-584cff5305eb       Abram53 Conrad619   
3       1045  09670eb9-5b42-a3ea-19d1-8b9cbe1b7643                 Alma679   
4       1130  0b5b9359-7147-3662-cd95-dfb09b858e19   Rosendo998 Augustus49   
5       1079  0d375b2a-e22f-a6ed-c4e9-a1a9790aebaa      Markus389 Loren192   
6       1099  0d463b76-c6c2-9fed-08a6-6258b83b8180     Gavin481 Garrett899   
7       1096  0d5f7ff0-0652-ba51-9c12-369d3159fe63    Nelson403 Sherman440   
8       1047  0fc6af72-bf86-5d2e-6591-875fb9f3861f              Julissa825   
9       1127  1053c422-40b5-520d-5a0e-cf3d1ea2309c   Vincenzo126 Carter549   
10      1139  1b446137-5e27-5511-f581-ca85af8215d4  Veronique514 Isabell86   
11      1152  1d09d64a-664c-dbac-4497-5ba18b3fccf3     Brian582 

In [3]:
# ML train/validate demo
# This cell prepares training data from the gene-level table and runs a small sklearn pipeline.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score

# load gene-level features created earlier
conn = sqlite3.connect('data/epic_synth.db')
vg = pd.read_sql('select GeneSymbol, pathogenic_count as pathogenic_count from (select GeneSymbol, count(*) as pathogenic_count from variant_pathogenic group by GeneSymbol)', conn)
# create numeric feature
X = vg[['pathogenic_count']].fillna(0)
y = (vg['pathogenic_count'] > 50).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000))
])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:,1]
print(classification_report(y_test, y_pred))
try:
    print('ROC AUC:', roc_auc_score(y_test, y_proba))
except Exception:
    pass

# save model to disk
import joblib
joblib.dump(pipe, 'data/gene_level_model.joblib')
print('model saved to data/gene_level_model.joblib')
conn.close()

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2017
           1       1.00      0.95      0.98       294

    accuracy                           0.99      2311
   macro avg       1.00      0.98      0.99      2311
weighted avg       0.99      0.99      0.99      2311

ROC AUC: 1.0
model saved to data/gene_level_model.joblib


# ETL Demo
Run the ETL to load FHIR JSON into SQLite and preview tables.

In [4]:
# Example: run the ETL process on data/fhir -> data/epic_synth.db
from epic_etl import run_etl
run_etl.process_fhir_dir('data/fhir', 'data/epic_synth.db')
print('ETL completed')

ETL completed


In [5]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data/epic_synth.db')
print(pd.read_sql('SELECT * FROM person LIMIT 5', conn))

                              person_id              given_name  family_name  \
0  09670eb9-5b42-a3ea-19d1-8b9cbe1b7643                 Alma679     Kunde533   
1  2087502f-d58c-670a-0d38-f319d948f707  Marketta481 Nereida276    Renner328   
2  0fc6af72-bf86-5d2e-6591-875fb9f3861f              Julissa825  Hermiston71   
3  cb864ad2-cc4c-638f-41aa-143abf3b40d4                 Thad495     Borer986   
4  2e584b4a-b12f-83cb-ca94-aac121f53d30   Franklin857 Garret233    Cronin387   

   gender  birth_date  
0  female  2016-03-28  
1  female  1961-05-31  
2  female  1999-07-21  
3    male  1942-09-30  
4    male  1942-09-16  
